In [1]:
import os
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

import sys
sys.path.append("/data2/zzd/rl_llm/verl")

import torch
from megatron.core import parallel_state as mpu

torch.distributed.init_process_group(backend="nccl", init_method="env://")
mpu.initialize_model_parallel(
    tensor_model_parallel_size=1,
    pipeline_model_parallel_size=1,
    virtual_pipeline_model_parallel_size=None,
    pipeline_model_parallel_split_rank=None,
    use_sharp=False,
    context_parallel_size=1,
    expert_model_parallel_size=1,
    nccl_communicator_config_path=None,
)

In [7]:
from verl.models.mcore.util import preprocess_packed_seqs, postprocess_packed_seqs

batch_size = 2
seq_len = 32

input_ids = torch.randint(0, 100, (batch_size, seq_len))
attention_mask = torch.ones((batch_size, seq_len), dtype=torch.bool)
attention_mask[:, -20:] = False  

input_ids_pad, packed_seq_params = preprocess_packed_seqs(input_ids, attention_mask)

In [9]:
print(input_ids)
print(attention_mask)
print(input_ids_pad)
print(packed_seq_params)

tensor([[60, 88, 81, 31, 50,  4, 34, 63,  2, 78, 59, 49, 61, 52, 17,  0, 54, 73,
          6, 23, 80, 85, 12, 41, 37, 50, 76, 29, 15, 55, 99, 59],
        [52, 84, 62, 95, 14, 96, 55,  8, 28, 97, 27, 49, 34, 43, 50, 71,  8, 48,
         98, 36, 10, 71, 11, 94, 45, 77,  5,  3, 93, 54, 20, 32]])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False]])
tensor([[60, 88, 81, 31, 50,  4, 34, 63,  2, 78, 59, 49, 52, 84, 62, 95, 14, 96,
         55,  8, 28, 97, 27, 49]])
PackedSeqParams(qkv_format='thd', cu_seqlens_q=tensor([ 0, 12, 24]